In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
import numpy as np
import matplotlib
matplotlib.use('Agg') # Use a non-interactive backend
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shap



In [16]:
import pandas as pd
new_data_df = pd.read_csv("dataset.csv")
new_data_df.describe()

,id,product_A_sold_in_the_past,product_B_sold_in_the_past,product_A_recommended,product_A,product_C,product_D,cust_hitrate,cust_interactions,cust_contracts,opp_month,opp_old,competitor_Z,competitor_X,competitor_Y,cust_in_iberia,target_variable
count,35899.000000,35899.000000,3.589900e+04,35899.000000,3.589900e+04,35899.000000,35899.000000,3.589900e+04,3.589900e+04,35899.000000,3.589900e+04,35899.000000,35899.000000,35899.000000,35899.000000,35899.000000,35899.000000
mean,17950.000000,-0.000004,6.100448e-08,0.000002,-9.822001e-07,-0.000004,0.000001,-3.562773e-07,-2.897017e-08,-0.000001,-1.607287e-07,0.000004,0.268197,0.001978,0.205382,0.603109,0.470487
std,10363.292993,1.000015,1.000014e+00,1.000014,1.000014e+00,1.000014,1.000014,1.000014e+00,1.000014e+00,1.000014,1.000014e+00,1.000013,0.443027,0.044429,0.403986,0.489260,0.499135
min,1.000000,-0.259920,-3.479400e-01,-0.109700,-8.910000e-02,-0.023720,-0.042470,-1.166700e+00,-6.817000e-01,-0.349970,-1.414640e+00,-0.281850,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8975.500000,-0.259920,-3.479400e-01,-0.109700,-8.910000e-02,-0.023720,-0.042470,-9.302400e-01,-6.817000e-01,-0.349970,-8.238200e-01,-0.281850,0.000000,0.000000,0.000000,0.000000,0.000000
50%,17950.000000,-0.259920,-3.479400e-01,-0.109700,-8.910000e-02,-0.023720,-0.042470,-2.187200e-01,-4.317300e-01,-0.349970,-2.330000e-01,-0.281850,0.000000,0.000000,0.000000,1.000000,0.000000
75%,26924.500000,-0.259920,-2.253200e-01,-0.109700,-8.910000e-02,-0.023720,-0.042470,8.437200e-01,3.616600e-01,-0.349970,9.486400e-01,-0.281850,1.000000,0.000000,0.000000,1.000000,1.000000
max,35899.000000,7.953860,8.546760e+00,25.323020,1.878559e+01,65.047500,36.231570,1.730920e+00,1.322972e+01,10.460370,1.834860e+00,3.547930,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
new_data_df

,product_A_sold_in_the_past,product_B_sold_in_the_past,product_A_recommended,product_A,product_C,product_D,cust_hitrate,cust_interactions,cust_contracts,opp_month,opp_old,competitor_Z,competitor_X,competitor_Y,cust_in_iberia
0,-0.25992,-0.34794,-0.1097,-0.0891,-0.02372,-0.04247,0.12643,0.30731,-0.34997,-1.41464,-0.28185,0,0,0,1


In [2]:
# --- Setup Output Directory ---
output_folder = 'rf_outputs'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Created directory: {output_folder}")

# Read data from dataset.csv
df = pd.read_csv('dataset.csv')

# Assuming the last column is the target variable and others are features
X = df.iloc[:, 1:-1]
y = df.iloc[:, -1]

# Split data into training, validation, and testing sets (80%, 10%, 10%)
# First, split into training (80%) and a temporary set (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# Then, split the temporary set into validation (10%) and testing (10%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# --- Hyperparameter Tuning (Pruning) ---
# Find the best max_depth using the validation set to prevent overfitting
# print("--- Finding Best Max Depth for Random Forest ---")
# depths = range(30, 40) 
# val_scores = []
# for depth in depths:
#     # Using n_estimators=100 is a common and robust starting point
#     model = RandomForestClassifier(n_estimators=100, max_depth=depth, random_state=42, n_jobs=-1)
#     model.fit(X_train, y_train)
#     val_scores.append(model.score(X_val, y_val))

best_depth = 35
print(f"Best max_depth found: {best_depth}")

# Initialize and fit the final model with the best hyperparameter
print("\n--- Training Final Random Forest Model ---")
model = RandomForestClassifier(n_estimators=65, max_depth=best_depth, random_state=42, min_samples_split=6, min_samples_leaf= 1, n_jobs=-1)
model.fit(X_train, y_train)

# --- Final Model Evaluation on the Test Set ---
print("\n--- Final Evaluation on Test Set ---")
y_pred = model.predict(X_test)

# Calculate and print key metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Model Accuracy on Test Set: {accuracy}")
print(f"F1 Score on Test Set: {f1}")



Best max_depth found: 35

--- Training Final Random Forest Model ---

--- Final Evaluation on Test Set ---
Model Accuracy on Test Set: 0.843175487465181
F1 Score on Test Set: 0.8432577771614429


In [3]:
df.head()

,id,product_A_sold_in_the_past,product_B_sold_in_the_past,product_A_recommended,product_A,product_C,product_D,cust_hitrate,cust_interactions,cust_contracts,opp_month,opp_old,competitor_Z,competitor_X,competitor_Y,cust_in_iberia,target_variable
0,1,-0.25992,-0.34794,-0.1097,-0.0891,-0.02372,-0.04247,0.12643,0.30731,-0.34997,-1.41464,-0.28185,0,0,0,1,1
1,2,-0.25992,-0.28599,-0.1097,-0.0891,-0.02372,-0.04247,0.42465,3.26349,0.92184,-1.11923,3.54793,1,0,0,0,0
2,3,-0.25992,-0.18998,-0.1097,-0.0891,-0.02372,-0.04247,0.72559,1.29633,-0.34997,0.65323,-0.28185,0,0,0,0,0
3,4,0.65945,3.35306,-0.1097,-0.0891,-0.02372,-0.04247,1.55091,-0.68170,3.46544,1.24404,-0.28185,0,0,0,1,0
4,5,-0.25992,-0.24623,-0.1097,-0.0891,-0.02372,-0.04247,-0.50193,0.65510,-0.03202,1.53945,-0.28185,1,0,0,0,0


In [3]:
# --- Model Interpretation and Explainability with SHAP ---

# 1. SHAP Feature Importance
print("\n--- Generating SHAP Feature Importance ---")
# For tree-based models like Random Forest, TreeExplainer is much more efficient
explainer = shap.TreeExplainer(model)

# # To speed up the calculation, we can explain a representative sample of the test data
# X_test_sample = shap.sample(X_test, 200, random_state=42)
# shap_values = explainer.shap_values(X_test_sample)

# plt.figure()
# # Note: We use the same sampled data for plotting
# shap.summary_plot(shap_values, X_test_sample, feature_names=X_test.columns.tolist(),
#     max_display=20,  # Show top 20 most important features
#     show=False)
# plt.title('SHAP Feature Importance for Random Forest')
# plt.savefig(os.path.join(output_folder, 'rf_shap_importance.png'), dpi=300)
# print(f"\nSaved SHAP feature importance plot to {output_folder}/rf_shap_importance.png")
# plt.close()

# explainer = shap.TreeExplainer(model)
 
shap.TreeExplainer(
    model, X_train, feature_names=X_train.columns.tolist()
)




--- Generating SHAP Feature Importance ---


In [6]:
rf_explainer = shap.TreeExplainer(
    model, X_train, feature_names=X_train.columns.tolist()
)
rf_explainer

In [ ]:
%%time
shap_values = rf_explainer.shap_values(X_train, y_train)

  7%|=                   | 3866/57438 [02:01<27:56]       

In [ ]:
shap_values = shap_values[:, :-1]

pd.DataFrame(shap_values, columns=X_train.columns.tolist()).head()

In [ ]:
shap.summary_plot(
    shap_values, X_train, feature_names=X_train.columns, plot_type="bar"
)

In [4]:
# 2. Standard Feature Importance
print("\n--- Feature Importance ---")
importances = model.feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

print(feature_importance_df)

# Plotting Feature Importance
plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='feature', data=feature_importance_df, palette='viridis')
plt.title('Feature Importance from Random Forest')
plt.xlabel('Importance Score')
plt.ylabel('Features')
plt.tight_layout()
plt.savefig(os.path.join(output_folder, 'rf_feature_importance.png'), dpi=300)
print(f"\nSaved feature importance plot to {output_folder}/rf_feature_importance.png")



--- Feature Importance ---
                       feature  importance
6                 cust_hitrate    0.324188
7            cust_interactions    0.191203
9                    opp_month    0.135442
1   product_B_sold_in_the_past    0.118977
0   product_A_sold_in_the_past    0.054755
8               cust_contracts    0.054115
10                     opp_old    0.034537
13                competitor_Y    0.019282
2        product_A_recommended    0.018506
14              cust_in_iberia    0.018105
11                competitor_Z    0.017960
3                    product_A    0.007887
5                    product_D    0.002732
12                competitor_X    0.001403
4                    product_C    0.000908


/tmp/ipykernel_67715/1388675107.py:12: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='importance', y='feature', data=feature_importance_df, palette='viridis')



Saved feature importance plot to rf_outputs/rf_feature_importance.png


In [5]:

# 3. Confusion Matrix
print("\n--- Confusion Matrix ---")
cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=model.classes_, yticklabels=model.classes_)
plt.title('Confusion Matrix for Random Forest')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.savefig(os.path.join(output_folder, 'rf_confusion_matrix.png'), dpi=300)
print(f"Saved confusion matrix plot to {output_folder}/rf_confusion_matrix.png")


--- Confusion Matrix ---
Saved confusion matrix plot to rf_outputs/rf_confusion_matrix.png
